In [23]:
# Setup
import pandas as pd
from pandas.testing import assert_series_equal
import datetime

pd.options.display.float_format = '{:.1f}'.format

# Assign A,B,C,... column names to source data frame.

df = pd.read_csv('./data/source.csv', na_values=[''])
questions: pd.Series = pd.read_csv('./config/questions.csv', index_col=0).squeeze()
col_names = questions.index.to_series()
questions_in_source = pd.Series(df.columns).replace(regex=r'\.[0-9]+$', value=r'')
assert_series_equal(questions, questions_in_source, check_names=False, check_index=False)

col_name_df = questions.copy()
col_name_df = col_names + ': ' + col_name_df

df.columns = col_names
df.index += 1
df.index.name = '回答番号'


# Find rows which is before start or declined
drop_candidates = df[((pd.to_datetime(df['A']).dt.tz_localize('+09:00')) <= datetime.datetime.fromisoformat('2024-11-01T00:00:00+09:00')) | (df['B'] != '了承して、回答する')]
drop_candidates.columns = col_names + ': ' + questions
drop_candidates.to_csv('./out/drop_candidates.csv')

# Find rows which does not meet requirement
requirement_drop_candidates = df[df['C'] != 'はい']
requirement_drop_candidates.columns = col_names + ': ' + questions
requirement_drop_candidates.to_csv('./out/requirement_drop_candidates.csv')


ac_drop_df = pd.read_csv('./data/after_codings/drops.csv')
drop_index = pd.Index(ac_drop_df['回答番号']).sort_values()
drop_df = df.loc[drop_index, :]
df.drop(drop_index, inplace=True)
assert(df['B'].unique().tolist() == ['了承して、回答する'])

# Convert float to int with NA
for column in df:
    if df[column].dtype.kind == 'f':
        df[column] = df[column].astype('Int64')

with_column_names_df = df.copy()
with_column_names_df.columns = col_names + ': ' + questions


In [24]:
# Setup choices_df
# choices_df contains multiple candidates of choice indexed by column name
choices_df = pd.read_csv('./config/choices.csv')
assert_series_equal(choices_df['設問文章'], questions[questions != "タイムスタンプ"], check_names=False, check_index=False)
choices_df.index = col_names.drop(index=questions[questions == "タイムスタンプ"].index).values
choices_df['選択肢'] = choices_df['選択肢'].str.split(',')


In [25]:
# Output raw data with modified column names

import os

os.makedirs('out/images', 0o755, exist_ok=True)

with_column_names_df.to_csv('out/raw.csv')


In [35]:
# Explode multiple choices into list, extract other answers

other_answer_to_indices_dict_by_col = { col: {} for col in choices_df[choices_df['その他回答'] == 1].index }

# single choice cols
for col in choices_df.loc[(choices_df['複数回答'] != 1) & (choices_df['その他回答'] == 1), :].index:
    other_answer_to_indices_dict_by_col[col] = df.groupby(col).apply(lambda x: list(x.index)).drop(choices_df.loc[col, '選択肢'], errors='ignore').to_dict()

# multi choice cols, with list
multi_choice_answers_df = df[choices_df.loc[choices_df['複数回答'] == 1, :].index].copy()
for col in multi_choice_answers_df.columns:
    choices_set = frozenset(choices_df.loc[col, '選択肢'])

    def split_choices(row):
        text = row.values[0]
        if not isinstance(text, str):
            return []
        attrs = text.split(', ')
        ret = []
        for i, attr in enumerate(attrs):
            if attr in choices_set:
                ret.append(attr)
            else:
                # Join texts after first other answer text (users can input exact ', ')
                rem = ', '.join(attrs[i:])
                ret.append(rem)
                dic = other_answer_to_indices_dict_by_col[col]
                dic.setdefault(rem, [])
                dic[rem].append(row.name)
                break
        return ret

    multi_choice_answers_df[col] = multi_choice_answers_df[col].to_frame().apply(split_choices, axis='columns')

col_other_answer_and_count_tuple_tuple = map(lambda col_dict: (col_dict[0], sorted(col_dict[1].items(), key=lambda t: len(t[1]), reverse=True)), other_answer_to_indices_dict_by_col.items())

out_other_answers_df = pd.DataFrame(col_other_answer_and_count_tuple_tuple, columns=['列番号', 'group']).explode('group').reset_index()
out_other_answers_df = pd.concat([out_other_answers_df['列番号'], pd.DataFrame(map(lambda item: (item[0], len(item[1]), item[1]), out_other_answers_df['group'].tolist()), columns=['回答文字列', '出現回数', '回答番号'])], axis='columns')
out_other_answers_df = out_other_answers_df.explode('回答番号')
out_other_answers_df.insert(1, '設問文章', choices_df.loc[out_other_answers_df['列番号'], '設問文章'].values)
out_other_answers_all_df = pd.merge(out_other_answers_df, with_column_names_df, how='left', left_on='回答番号', right_index=True)
out_other_answers_all_df.to_csv('out/other_answers.csv', index=False)

os.makedirs('out/other_answers', 0o755, exist_ok=True)

for col, group in out_other_answers_df.groupby('列番号'):
    question: str = questions[col]
    normalized_question = question.replace('/', '_')
    # TODO: 選択肢一覧をつける
    group.to_csv(f'out/other_answers/{col}_{normalized_question}.csv', index=False)


In [27]:
from typing import List

# Find duplicated

dup_groups_list: List[pd.DataFrame] = []

free_text_cols = choices_df[choices_df['自由記述'] == 1].index
non_free_text_cols = choices_df.index.drop(free_text_cols)

dup_all_rows_df = df[df.duplicated(non_free_text_cols, keep=False)][non_free_text_cols].dropna(how='all')
dup_all_rows_df.sort_values(non_free_text_cols.tolist())
dup_all_group_df = dup_all_rows_df.iloc[:, 0:0].copy()
dup_all_group_df['回答'] = '自由記述以外のすべて'
dup_all_group_df.insert(0, '列番号', '自由記述以外のすべて')
dup_all_group_df.reset_index(inplace=True)
dup_groups_list.append(dup_all_group_df)

dup_unique_df = df.drop(dup_all_rows_df.index)

for col in free_text_cols:
    dup_series: pd.Series = dup_unique_df[dup_unique_df.duplicated(col, keep=False)][col].dropna(how='all')
    dup_groups = dup_series.sort_values()
    dup_group_df = dup_groups.to_frame()
    dup_group_df.columns = ['回答']
    dup_group_df.insert(0, '列番号', col)
    dup_group_df.reset_index(inplace=True)
    dup_groups_list.append(dup_group_df)

out_dup_df = pd.concat(dup_groups_list, ignore_index=True)
out_dup_df = pd.merge(out_dup_df, with_column_names_df, how='left', left_on='回答番号', right_index=True)
out_dup_df.to_csv('out/duplicates.csv', index=False)

In [28]:
# Exclusive choices


excl_config_df = pd.read_csv('./config/exclusive_choices.csv')

other_answer_keyword = "他の選択肢"

excl_entry_list = []

for index, row in excl_config_df.iterrows():
    col = row['設問番号']
    before_choice = row['他の選択肢と押すと矛盾する選択肢']
    if pd.isna(before_choice):
        continue
    choice_list = choices_df.loc[col, '選択肢']
    if not before_choice in choice_list:
        raise Exception(f"\"{before_choice}\" is not a valid choice, valid choices: {choice_list}")

    matched_series = df.loc[multi_choice_answers_df[col].apply(lambda arr: before_choice in arr and len(arr) >= 2), col]

    if row['残す選択肢'] == other_answer_keyword:
        after_series = multi_choice_answers_df.loc[matched_series.index, col].apply(lambda arr: ','.join([x for x in arr if x == before_choice]))
    else:
        after_series = [row['残す選択肢']] * len(matched_series)

    excl_entry_df = pd.DataFrame({
        '回答番号': matched_series.index,
        '設問番号': [col] * len(matched_series),
        '他の選択肢と押すと矛盾する選択肢': [before_choice] * len(matched_series),
        '変更前': matched_series.values,
        '変更後': after_series
    })
    excl_entry_list.append(excl_entry_df)

if len(excl_entry_list) > 0:
    out_excl_df = pd.concat(excl_entry_list, ignore_index=True)
    out_excl_df.to_csv('out/exclusives.csv', index=False)
else:
    out_excl_df = pd.DataFrame(columns=['回答番号', '設問番号', '他の選択肢と押すと矛盾する選択肢', '変更前', '変更後'])
    out_excl_df.to_csv('out/exclusives.csv', index=False)

In [29]:
# Detect age contradictions

current_age_column = 'D'
# TODO: 設定ファイルにする
answer_age_columns = ['AK', 'AL', 'AO']

has_age_contradictions_df = df[answer_age_columns].gt(df[current_age_column], axis='index')

out_age_contra_df = has_age_contradictions_df[has_age_contradictions_df.any(axis='columns')].copy()
out_age_contra_df = df.loc[out_age_contra_df.index, answer_age_columns][out_age_contra_df]
out_age_contra_df.insert(0, 'D', df.loc[out_age_contra_df.index, current_age_column])
# TODO: add question text to columns
out_age_contra_df.columns = col_name_df.loc[[current_age_column] + answer_age_columns].values
out_age_contra_df.astype('Int64').to_csv('out/age_contradictions.csv')


In [30]:
# After codings

import itertools
import re


ac_age_df = pd.read_csv('./data/after_codings/age_contradictions.csv', keep_default_na=False, na_values=[''])

ac_age_d_df = ac_age_df[['回答番号', 'D', 'Dの変更後']].copy()
ac_age_d_df.columns = ['回答番号', '修正前', '修正後']
ac_age_d_df.dropna(subset='修正後', inplace=True)
ac_age_d_df[['修正前', '修正後']] = ac_age_d_df[['修正前', '修正後']].replace('NA', None).astype('float64')
ac_age_d_df.insert(0, '列番号', 'D')

ac_age_at_df = ac_age_df[['回答番号', 'AT', 'ATの変更後']].copy()
ac_age_at_df.columns = ['回答番号', '修正前', '修正後']
ac_age_at_df.dropna(subset='修正後', inplace=True)
ac_age_at_df[['修正前', '修正後']] = ac_age_at_df[['修正前', '修正後']].replace('NA', None).astype('float64')
ac_age_at_df.insert(0, '列番号', 'AT')

ac_exclusives_df = pd.read_csv('./data/after_codings/exclusives.csv')[['列番号', '回答番号', '回答', '変更後']]
ac_exclusives_df.columns = ['列番号', '回答番号', '修正前', '修正後']
ac_exclusives_df['修正前'] = ac_exclusives_df['修正前'].str.split(', ')
ac_exclusives_df['修正後'] = ac_exclusives_df['修正後'].str.split(r', ?', regex=True)


q_df = questions.reset_index()
q_df.columns = ['列番号', '設問文章']
ac_transforms_df = pd.read_csv('./data/after_codings/transforms.csv').merge(q_df, how='left', on='設問文章', copy=False)[['列番号', '回答番号', '修正前', '修正後']]

ac_others_df = pd.read_csv('./data/after_codings/others.csv')[['列番号', '回答番号', '回答文字列', '変更後の回答']]
ac_others_df.columns = ['列番号', '回答番号', '修正前', '修正後']
# split string for multiple answers questions
ac_others_df.update(ac_others_df[ac_others_df.merge(choices_df['複数回答'], how='left', left_on='列番号', right_index=True)['複数回答'] == 1]['修正後'].str.split(re.compile(r', ?')))

ac_data_df = df.copy()
ac_multi_choice_df = multi_choice_answers_df.copy()

ac_changes = pd.concat([ac_age_d_df, ac_age_at_df, ac_exclusives_df, ac_transforms_df, ac_others_df])
for column, group in ac_changes.groupby('列番号', sort=False):
    if column in ac_multi_choice_df.columns:
        has_comma_series = group['修正後'].str.join('').str.contains(',')
        if has_comma_series.any():
            raise ValueError(f'{column} has translation cell with untreated comma: {group[has_comma_series]["修正後"]}')

        for _, row in group.iterrows():
            before_list = ac_multi_choice_df.loc[row['回答番号'], column]
            if isinstance(row['修正前'], list):
                if before_list != row['修正前']:
                    raise ValueError(f'element {row["修正前"]} does not match for index {row["回答番号"]} of column {column} (actual {before_list}, translating to {row["修正後"]})')
                ac_multi_choice_df.loc[row['回答番号'], column] = row['修正後']
            else:
                if row['修正前'] not in map(str.strip, before_list):
                    raise ValueError(f'element {row["修正前"]} not found for index {row["回答番号"]} of column {column} (actual {before_list}, translating to {row["修正後"]})')
                ac_multi_choice_df.loc[row['回答番号'], column] = list(dict.fromkeys(
                    itertools.chain.from_iterable([row['修正後'] if x.strip() == row['修正前'] else [x] for x in before_list])
                ))
    else:
        change_set = group.set_index('回答番号')
        if change_set.index.has_duplicates:
            duplicates = ', '.join(map(str, change_set.index[change_set.index.duplicated()]))
            raise ValueError(f'Column {column} has duplicated changes for same index ({duplicates})')
        assert_series_equal(ac_data_df[column][group['回答番号']], change_set['修正前'], check_names=False, check_dtype=False)
        ac_data_df.loc[change_set.index, column] = change_set['修正後']

# Remove rows which does not meet requirement
ac_requirement_drop_index =  ac_data_df[ac_data_df['C'] == 'いいえ'].index
ac_data_df.drop(ac_requirement_drop_index, inplace=True)
assert(ac_data_df['C'].unique().tolist() == ['はい'])

# Strip text in free text answers
for col in choices_df[(choices_df['その他回答'] == 1) | (choices_df['自由記述'] == 1)].index:
    if ac_data_df[col].dtype != 'object':
        print(f"Warning: column {col} is not object type. Maybe empty free text column?")
    else:
        ac_data_df[col] = ac_data_df[col].str.strip(' 　')
for col in ac_multi_choice_df:
    ac_multi_choice_df[col] = ac_multi_choice_df[col].apply(lambda arr: list(map(lambda s: s.strip(' 　'), arr)))

out_ac_df = ac_data_df.copy()
for column, series in ac_multi_choice_df.items():
    out_ac_df[column] = series.str.join(', ')
assert_series_equal(out_ac_df.columns.to_series(), col_names)
out_ac_df.columns = col_names + ': ' + questions
out_ac_df.to_csv('out/after_coded.csv')

out_ac_serialized_df = ac_data_df.copy()
out_ac_serialized_df.update(ac_multi_choice_df)
assert_series_equal(out_ac_serialized_df.columns.to_series(), col_names)
out_ac_serialized_df.reset_index(inplace=True)
out_ac_serialized_df.to_feather('out/after_coded.feather')


/var/folders/5g/gdhb1py554jdy7kh5f89k0mw0000gn/T/ipykernel_78445/1046892204.py:81: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in ac_multi_choice_df.iteritems():


In [31]:
out_drop_df = pd.concat([drop_df, df.loc[ac_requirement_drop_index, :]]).sort_index()
out_drop_df.columns = col_names + ': ' + questions
out_drop_df.to_csv('out/drop.csv')
